### Data Preprocessing

#### Data Cleaning and Aggregation

In [1]:
import numpy as np
import pandas as pd

# Load raw datasets
df_wave = pd.read_csv('WLIS_wave_raw.csv')
df_wind = pd.read_csv('WLIS_wind_raw.csv')

# Set the timestamp column as the index
df_wave['TmStamp'] = pd.to_datetime(df_wave['TmStamp'], format='mixed')
df_wind['TmStamp'] = pd.to_datetime(df_wind['TmStamp'], format='mixed')
df_wave.set_index('TmStamp', inplace=True)
df_wind.set_index('TmStamp', inplace=True)

# Rename columns
df_wave = df_wave.rename(columns={'Hsig_m':'H'})
df_wind = df_wind.rename(columns={'windSpd_Kts':'WSPD', 'windDir_M':'WDIR'})

# In total 118,872 hourly timestamps
date_ranges = [
    ('2004-11-01 00:00:00', '2015-01-05 23:00:00'),  # 89232 hourly timestamps
    ('2016-12-12 00:00:00', '2018-01-11 23:00:00'),  # 9504 hourly timestamps
    ('2018-04-27 00:00:00', '2018-10-22 23:00:00'),  # 4296 hourly timestamps
    ('2019-05-28 00:00:00', '2019-12-31 23:00:00'),  # 5232 hourly timestamps
    ('2022-12-09 00:00:00', '2023-06-02 23:00:00'),  # 4224 hourly timestamps
    ('2024-03-28 00:00:00', '2024-12-18 23:00:00')]  # 6384 hourly timestamps

# Filter date ranges
ndf_wave = pd.DataFrame()
ndf_wind = pd.DataFrame()
date_ranges[0] = ('2006-02-23 00:00:00', '2015-01-05 23:00:00')
for start, end in [(pd.to_datetime(start), pd.to_datetime(end[:-5] + '59:59')) for start, end in date_ranges]:
    ndf_wave = pd.concat([ndf_wave, df_wave[(df_wave.index >= start) & (df_wave.index <= end)]])
    ndf_wind = pd.concat([ndf_wind, df_wind[(df_wind.index >= start) & (df_wind.index <= end)]])

# Remove outliers
ndf_wind.loc[ndf_wind['WSPD'] > 50, 'WSPD'] = np.nan

# Resample by hour
ndf_wave = ndf_wave.resample('h').max()
ndf_wind = ndf_wind.resample('h').max()

# Convert units
ndf_wave['H'] = 3.28084 * ndf_wave['H']         # m to ft
ndf_wind['WSPD'] = 0.514444 * ndf_wind['WSPD']  # kts to m/s

# Fill in the missing timestamps
timestamps = []
date_ranges[0] = ('2004-11-01 00:00:00', '2015-01-05 23:00:00')
for start, end in [(pd.to_datetime(start), pd.to_datetime(end)) for start, end in date_ranges]:
    timestamps.extend(pd.date_range(start, end, freq='h'))

# Merge datasets
df_merge = pd.merge(ndf_wave, ndf_wind, how='outer', left_index=True, right_index=True)
df_merge = pd.merge(pd.DataFrame(timestamps, columns=['TmStamp']), df_merge, on='TmStamp', how='left')
df_merge = df_merge.sort_values('TmStamp').reset_index(drop=True)

#### Data Imputation Using 2004-2013 Buoy Dataset

In [2]:
# Load 2004-2013 buoy dataset
df_buoy = pd.read_csv('Data_Buoy_2004_2013.csv')
df_buoy = df_buoy.rename(columns={'TimeStamp_1':'TmStamp', 'WSPD':'WSPD1', 'WDIR':'WDIR1'})
df_buoy['TmStamp'] = pd.to_datetime(df_buoy['TmStamp'], format='mixed')

# First imputation
df_new = pd.merge(df_merge, df_buoy[['TmStamp','SWHft','WSPD1','WDIR1']], on='TmStamp', how='left')
df_new['H'] = df_new['H'].fillna(df_new['SWHft'])
df_new['WSPD'] = df_new['WSPD'].fillna(df_new['WSPD1'])
df_new['WDIR'] = df_new['WDIR'].fillna(df_new['WDIR1'])
df_new = df_new[['TmStamp','H','WSPD','WDIR']]

#### Wind Data Imputation

In [3]:
# Count consecutive NaN lengths
def get_nan_lengths(series):
    nans = series.isna()
    group = (nans != nans.shift()).cumsum()
    nan_lengths = nans.groupby(group).transform('sum')
    return np.where(nans, nan_lengths, 0).astype(int)

# Case 1: Interpolate when nan_lengths ≤ 5
WDIR = df_new['WDIR'].interpolate(method='linear')
WSPD = df_new['WSPD'].interpolate(method='linear')
n_random = np.random.normal(loc=0, scale=1, size=len(df_new))
sd_WSPD = np.std(df_new['WSPD'].dropna())
WSPD += n_random * sd_WSPD

WDIR_nans = get_nan_lengths(df_new['WDIR'])
WSPD_nans = get_nan_lengths(df_new['WSPD'])

WDIR_mask = (WDIR_nans > 0) & (WDIR_nans <= 5)
WSPD_mask = (WSPD_nans > 0) & (WSPD_nans <= 5)

df_new.loc[WDIR_mask, 'WDIR'] = WDIR[WDIR_mask]
df_new.loc[WSPD_mask, 'WSPD'] = WSPD[WSPD_mask]

# Case 2: Sample from known ('WSPD', 'WDIR') when nan_lengths > 5
df_new['season'] = np.where(df_new['TmStamp'].dt.month.isin([11, 12, 1, 2, 3]), 'windy', 'calm')
for season in ['windy','calm']:
    df_season = df_new[df_new['season'] == season]
    mask = (WSPD_nans > 5) & (df_new['season'] == season)
    sampled = df_season[['WSPD','WDIR']].dropna().sample(n=mask.sum(), replace=True)
    df_new.loc[mask, ['WSPD','WDIR']] = sampled.values

#### Wave Data Imputation

In [4]:
# Create lagged WSPD quartile bins
df_new['rWSPD'] = df_new['WSPD'].shift(1)
quartiles = df_new['rWSPD'].quantile([0.25, 0.5, 0.75])
df_new['bin'] = pd.cut(
    df_new['rWSPD'],
    bins = [-np.inf, quartiles[0.25], quartiles[0.5], quartiles[0.75], np.inf],
    labels = ['Q1','Q2','Q3','Q4'])

# Impute NaN with the median wave height for each bin categorized by season
damp = {'windy':0.03, 'calm':0.01}
for season in ['windy','calm']:
    df_season = df_new[df_new['season'] == season]
    for label in ['Q1','Q2','Q3','Q4']:
        mask = (df_new['H'].isna()) & (df_new['bin'] == label) & (df_new['season'] == season)
        obs_median = df_season[df_season['bin'] == label]['H'].median()
        n_random = np.random.normal(loc=0, scale=1, size=mask.sum())
        df_new.loc[mask, 'H'] = obs_median + damp[season] * n_random

#### Fetch Calculation & Processed Dataset Output

In [5]:
fetch_data = [
    (0, 3), (10, 5), (20, 5), (30, 10), (40, 12), (50, 17), (60, 40), (70, 75),
    (80, 61), (90, 22), (100, 17), (110, 11), (120, 4), (130, 4), (140, 3), (150, 3),
    (160, 3), (170, 3), (180, 3), (190, 3), (200, 3), (210, 4), (220, 5), (230, 10),
    (240, 14), (250, 11), (260, 8), (270, 7), (280, 6), (290, 5), (300, 5), (310, 5),
    (320, 5), (330, 5), (340, 5), (350, 4), (360, 3)]

fetch_df = pd.DataFrame(fetch_data, columns=['Deg','Dist'])
df_new['Fetch'] = np.interp(df_new['WDIR'], fetch_df['Deg'], fetch_df['Dist'])
df_new['Fetch'] = 1609.34 * df_new['Fetch']

# Save the processed dataset
df_new = df_new[['TmStamp','H','WSPD','WDIR','Fetch']]
df_new = df_new.sort_values('TmStamp').reset_index(drop=True)
df_new.to_csv('WLIS_data.csv', index=False)

df_new.head()

,TmStamp,H,WSPD,WDIR,Fetch
0,2004-11-01 00:00:00,3.249133,13.4,120.0,6437.36
1,2004-11-01 01:00:00,3.249133,8.2,80.0,98169.74
2,2004-11-01 02:00:00,1.960359,7.7,340.0,8046.70
3,2004-11-01 03:00:00,2.001249,10.3,10.0,8046.70
4,2004-11-01 04:00:00,3.254957,11.8,50.0,27358.78
